In [7]:
# Importar librería para la conexión con MySQL
# -----------------------------------------------------------------------
import mysql.connector
from mysql.connector import errorcode


# Importar librerías para manipulación y análisis de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

In [37]:
df_rawdata = pd.read_csv("../Data/raw_data_final_20231212.csv", index_col=0) 
df_rawdata.head(2)

,Age,Attrition,Businesstravel,Department,Education,Educationfield,Employeenumber,Environmentsatisfaction,Gender,Jobinvolvement,...,Yearswithcurrmanager,Datebirth,Remotework,Dailyrate,Distancefromhome,Monthlyincome,Performancerating,Totalworkingyears,Worklifebalance,Hourlyrate
0,51,No,Unknown,Research & Development,3,Unknown,162,Not Satisfied,Male,High,...,15,1972,Yes,684.0,6.0,19537.0,3.0,13.6,3.0,51.0
1,52,No,Unknown,Unknown,4,Life Sciences,259,Satisfied,Male,Moderate,...,9,1971,Yes,699.0,1.0,19999.0,3.0,34.0,3.0,65.0


# TABLA 1 Ficha personal 

In [9]:
#DATA FRAME DE LA TABLA 1 - FICHA PERSONAL 
df_tabla1 = df_rawdata[["Employeenumber","Age", "Gender", "Maritalstatus", "Datebirth"]]
df_tabla1.head(3)

,Employeenumber,Age,Gender,Maritalstatus,Datebirth
0,162,51,Male,Unknown,1972
1,259,52,Male,Unknown,1971
2,319,42,Male,Married,1981


In [10]:
lista_tuplas_tabla = []
for x in df_tabla1.values:

    lista_tuplas_tabla.append(tuple(x))
    lista_tuplas_tabla

In [11]:
cnx = mysql.connector.connect (user = "root", 
                               password = "AlumnaAdalab", 
                               host='127.0.0.1',
                               database="talent_transformation")

mycursor = cnx.cursor()

query_insert = "INSERT INTO 1_personal_data (Employeenumber, Age, Gender, Maritalstatus, Datebirth) VALUES (%s, %s, %s, %s, %s)"
#variables en el mismo orden que scsv, tantas %s como variables tiene mi csv. 


try: 
    mycursor.executemany(query_insert, lista_tuplas_tabla)
    #ejecuta my query, de que datos? del fichero q estan en mi lista de tuplas q he converido ariba leyendo el csv.) 
    cnx.commit()
    print(mycursor.rowcount, "registros insertados")

except mysql.connector.Error as error:
    print("Error al insertar datos en MySQL:", error)

Error al insertar datos en MySQL: 1062 (23000): Duplicate entry '162' for key '1_personal_data.PRIMARY'


# TABLA 2 Ficha laboral 

In [12]:
# Cambiar el nombre de la columna "Yearswithcurrentmanager" a "Yearswithcurrmanager"
df_rawdata.rename(columns={"Yearswithcurrmanager": "Yearswithcurrentmanager"}, inplace=True)



In [13]:
df_rawdata

,Age,Attrition,Businesstravel,Department,Education,Educationfield,Employeenumber,Environmentsatisfaction,Gender,Jobinvolvement,...,Yearswithcurrentmanager,Datebirth,Remotework,Dailyrate,Distancefromhome,Monthlyincome,Performancerating,Totalworkingyears,Worklifebalance,Hourlyrate
0,51,No,Unknown,Research & Development,3,Unknown,162,Not Satisfied,Male,High,...,15,1972,Yes,684.0,6.0,19537.0,3.0,13.6,3.0,51.0
1,52,No,Unknown,Unknown,4,Life Sciences,259,Satisfied,Male,Moderate,...,9,1971,Yes,699.0,1.0,19999.0,3.0,34.0,3.0,65.0
2,42,No,travel_rarely,Research & Development,2,Technical Degree,319,Satisfied,Male,High,...,15,1981,Yes,532.0,4.0,19232.0,3.0,22.0,3.0,58.0
3,47,No,travel_rarely,Research & Development,4,Medical,2069,Not Satisfied,Female,High,...,6,1976,No,359.0,2.0,17169.0,3.0,11.2,1.8,82.0
4,46,No,Unknown,Sales,3,Technical Degree,2070,Not Satisfied,Female,Very High,...,8,1977,No,1319.0,3.0,4884.6,3.0,13.4,3.0,45.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1609,36,Yes,travel_rarely,Sales,1,Life Sciences,967,Satisfied,Male,Moderate,...,7,1987,No,530.0,3.0,10325.0,3.4,16.8,3.0,51.0
1610,45,No,non-travel,Research & Development,2,Unknown,972,Satisfied,Male,High,...,8,1978,Yes,805.0,4.0,4447.0,3.0,14.6,2.0,57.0
1611,39,No,travel_rarely,Sales,5,Unknown,2498,Not Satisfied,Male,Very High,...,7,1984,Yes,903.0,10.6,7144.8,3.0,9.0,3.0,41.0
1612,36,No,non-travel,Sales,4,Technical Degree,990,Not Satisfied,Male,High,...,7,1987,Yes,1229.0,8.0,5839.8,3.0,12.0,3.0,84.0


In [14]:
df_tabla2 = df_rawdata[[
    'Joblevel',
    'Jobrole',
    'Businesstravel',
    'Department',
    'Overtime',
    'Distancefromhome',
    'Yearssincelastpromotion',
    'Yearsatcompany',
    'Yearswithcurrentmanager',
    'Remotework',
    'Trainingtimeslastyear',
    'Attrition',
    'Employeenumber'
]]
df_tabla2.head(3)

,Joblevel,Jobrole,Businesstravel,Department,Overtime,Distancefromhome,Yearssincelastpromotion,Yearsatcompany,Yearswithcurrentmanager,Remotework,Trainingtimeslastyear,Attrition,Employeenumber
0,Executive,Research Director,Unknown,Research & Development,No,6.0,15,20,15,Yes,5,No,162
1,Executive,Manager,Unknown,Unknown,Unknown,1.0,11,33,9,Yes,5,No,259
2,Executive,Manager,travel_rarely,Research & Development,No,4.0,11,22,15,Yes,3,No,319


In [15]:
df_tabla_limpio = df_tabla2.fillna(0)

In [16]:
df_tabla_limpio

,Joblevel,Jobrole,Businesstravel,Department,Overtime,Distancefromhome,Yearssincelastpromotion,Yearsatcompany,Yearswithcurrentmanager,Remotework,Trainingtimeslastyear,Attrition,Employeenumber
0,Executive,Research Director,Unknown,Research & Development,No,6.0,15,20,15,Yes,5,No,162
1,Executive,Manager,Unknown,Unknown,Unknown,1.0,11,33,9,Yes,5,No,259
2,Executive,Manager,travel_rarely,Research & Development,No,4.0,11,22,15,Yes,3,No,319
3,Senior,Research Director,travel_rarely,Research & Development,Unknown,2.0,5,20,6,No,2,No,2069
4,Senior,Sales Executive,Unknown,Sales,No,3.0,2,19,8,No,5,No,2070
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1609,Mid-level,Sales Executive,travel_rarely,Sales,Unknown,3.0,3,16,7,No,6,Yes,967
1610,Junior,Laboratory Technician,non-travel,Research & Development,Unknown,4.0,0,9,8,Yes,5,No,972
1611,Mid-level,Sales Executive,travel_rarely,Sales,No,10.6,0,8,7,Yes,3,No,2498
1612,Junior,Sales Executive,non-travel,Sales,No,8.0,0,7,7,Yes,3,No,990


In [17]:
lista_tuplas_tabla = []
for x in df_tabla_limpio.values:

    lista_tuplas_tabla.append(tuple(x))
    lista_tuplas_tabla

In [18]:
cnx = mysql.connector.connect (user = "root", 
                               password = "AlumnaAdalab", 
                               host='127.0.0.1',
                               database="talent_transformation")

mycursor = cnx.cursor()

query_insert = "INSERT INTO 2_laboral_data (Joblevel,Jobrole,Businesstravel,Department,Overtime,Distancefromhome,Yearssincelastpromotion,Yearsatcompany,Yearswithcurrentmanager,Remotework,Trainingtimeslastyear,Attrition,1_personal_data_Employeenumber) VALUES (%s, %s, %s, %s, %s,%s, %s, %s, %s, %s,%s, %s, %s)"
#variables en el mismo orden que scsv, tantas %s como variables tiene mi csv. 


try: 
    mycursor.executemany(query_insert, lista_tuplas_tabla)
    #ejecuta my query, de que datos? del fichero q estan en mi lista de tuplas q he converido ariba leyendo el csv.) 
    cnx.commit()
    print(mycursor.rowcount, "registros insertados")

except mysql.connector.Error as error:
    print("Error al insertar datos en MySQL:", error)

1510 registros insertados


# TABLA 2B laboral life 

In [19]:
df_rawdata.columns

Index(['Age', 'Attrition', 'Businesstravel', 'Department', 'Education',
       'Educationfield', 'Employeenumber', 'Environmentsatisfaction', 'Gender',
       'Jobinvolvement', 'Joblevel', 'Jobrole', 'Jobsatisfaction',
       'Maritalstatus', 'Monthlyrate', 'Numcompaniesworked', 'Overtime',
       'Percentsalaryhike', 'Relationshipsatisfaction', 'Stockoptionlevel',
       'Trainingtimeslastyear', 'Yearsatcompany', 'Yearssincelastpromotion',
       'Yearswithcurrentmanager', 'Datebirth', 'Remotework', 'Dailyrate',
       'Distancefromhome', 'Monthlyincome', 'Performancerating',
       'Totalworkingyears', 'Worklifebalance', 'Hourlyrate'],
      dtype='object')

In [20]:
df_tabla2B = df_rawdata[["Numcompaniesworked","Totalworkingyears", "Education", "Educationfield", "Employeenumber"]]
df_tabla2B.head(3)



,Numcompaniesworked,Totalworkingyears,Education,Educationfield,Employeenumber
0,7,13.6,3,Unknown,162
1,0,34.0,4,Life Sciences,259
2,1,22.0,2,Technical Degree,319


In [21]:
lista_tuplas_tabla = []
for x in df_tabla2B.values:

    lista_tuplas_tabla.append(tuple(x))
    lista_tuplas_tabla

In [22]:
cnx = mysql.connector.connect (user = "root", 
                               password = "AlumnaAdalab", 
                               host='127.0.0.1',
                               database="talent_transformation")

mycursor = cnx.cursor()

query_insert = "INSERT INTO 2B_laboral_life (Numcompaniesworked, Totalworkingyears, Education, Educationfield, 1_personal_data_Employeenumber) VALUES (%s, %s, %s, %s, %s)"
#variables en el mismo orden que scsv, tantas %s como variables tiene mi csv. 


try: 
    mycursor.executemany(query_insert, lista_tuplas_tabla)
    #ejecuta my query, de que datos? del fichero q estan en mi lista de tuplas q he converido ariba leyendo el csv.) 
    cnx.commit()
    print(mycursor.rowcount, "registros insertados")

except mysql.connector.Error as error:
    print("Error al insertar datos en MySQL:", error)

1510 registros insertados


# TABLA 3 economic life -- Pending

In [23]:
df_rawdata.columns

Index(['Age', 'Attrition', 'Businesstravel', 'Department', 'Education',
       'Educationfield', 'Employeenumber', 'Environmentsatisfaction', 'Gender',
       'Jobinvolvement', 'Joblevel', 'Jobrole', 'Jobsatisfaction',
       'Maritalstatus', 'Monthlyrate', 'Numcompaniesworked', 'Overtime',
       'Percentsalaryhike', 'Relationshipsatisfaction', 'Stockoptionlevel',
       'Trainingtimeslastyear', 'Yearsatcompany', 'Yearssincelastpromotion',
       'Yearswithcurrentmanager', 'Datebirth', 'Remotework', 'Dailyrate',
       'Distancefromhome', 'Monthlyincome', 'Performancerating',
       'Totalworkingyears', 'Worklifebalance', 'Hourlyrate'],
      dtype='object')

In [53]:
df_tabla3["Hourlyrate"].isnull().value_counts()

Hourlyrate
False    1510
Name: count, dtype: int64

In [46]:
df_tabla3 = df_rawdata[["Monthlyrate","Percentsalaryhike", "Stockoptionlevel", "Dailyrate", "Monthlyincome", "Hourlyrate", "Employeenumber"]]
df_tabla3.head(3)

,Monthlyrate,Percentsalaryhike,Stockoptionlevel,Dailyrate,Monthlyincome,Hourlyrate,Employeenumber
0,6462,13,0,684.0,19537.0,51.0,162
1,5678,14,1,699.0,19999.0,65.0,259
2,4933,11,0,532.0,19232.0,58.0,319


In [47]:
lista_tuplas_tabla = []
for x in df_tabla3.values:

    lista_tuplas_tabla.append(tuple(x))
    lista_tuplas_tabla

In [59]:
lista_tuplas_tabla[1000:]

[(4167.0, 18.0, 3.0, 528.0, 5361.8, 44.0, 1683.0),
 (23737.0, 12.0, 1.0, 1441.0, 9881.8, 94.0, 1694.0),
 (25103.0, 21.0, 1.0, 427.0, 5844.2, 99.0, 1702.0),
 (6217.0, 15.0, 3.0, 390.0, 2305.0, 62.0, 2358.0),
 (17360.0, 13.0, 1.0, 585.0, 3433.0, 65.6, 2359.0),
 (18103.0, 14.0, 1.0, 741.0, 3477.0, 69.0, 1721.0),
 (20794.0, 19.0, 1.0, 552.0, 4709.4, 60.0, 1722.0),
 (3622.0, 13.0, 3.0, 506.0, 3907.0, 91.0, 2360.0),
 (8658.0, 13.0, 3.0, 1456.0, 10477.4, 96.0, 1733.0),
 (12862.0, 13.0, 0.0, 160.0, 3325.2, 51.0, 1735.0),
 (2097.0, 14.0, 1.0, 897.0, 3367.4, 59.0, 1746.0),
 (9732.0, 11.0, 1.0, 600.0, 4847.6, 68.2, 2361.0),
 (20943.0, 19.0, 1.0, 1003.0, 3505.2, 51.0, 2362.0),
 (6152.0, 11.0, 0.0, 1054.0, 3445.0, 67.0, 1751.0),
 (14630.0, 13.0, 0.0, 428.0, 2760.0, 66.8, 2363.0),
 (23060.0, 12.0, 0.0, 461.0, 6294.0, 70.0, 1753.0),
 (22308.0, 20.0, 3.0, 661.0, 6814.4, 60.0, 2364.0),
 (25353.0, 12.0, 0.0, 511.0, 5578.4, 32.0, 2365.0),
 (22984.0, 11.0, 0.0, 942.0, 13116.0, 53.0, 2366.0),
 (8800.0, 14.

In [58]:
lista_tuplas_tabla[0:1000]

[(6462.0, 13.0, 0.0, 684.0, 19537.0, 51.0, 162.0),
 (5678.0, 14.0, 1.0, 699.0, 19999.0, 65.0, 259.0),
 (4933.0, 11.0, 0.0, 532.0, 19232.0, 58.0, 319.0),
 (26703.0, 19.0, 2.0, 359.0, 17169.0, 82.0, 2069.0),
 (7739.0, 12.0, 1.0, 1319.0, 4884.6, 45.0, 2070.0),
 (2437.0, 11.0, 1.0, 117.0, 17174.0, 58.0, 1900.0),
 (2354.0, 11.0, 0.0, 1435.0, 10477.4, 99.0, 81.0),
 (24532.0, 25.0, 0.0, 635.0, 8659.6, 99.0, 387.0),
 (5626.0, 16.0, 1.0, 1276.0, 16595.0, 91.0, 2071.0),
 (3735.0, 17.0, 1.0, 840.0, 7971.4, 64.0, 999.0),
 (20284.0, 22.0, 2.0, 247.0, 19973.0, 55.0, 1035.0),
 (18235.0, 12.0, 1.0, 1369.0, 13402.0, 68.0, 2072.0),
 (3376.0, 12.0, 1.0, 201.0, 13206.0, 99.0, 2073.0),
 (16280.0, 12.0, 0.0, 1360.0, 19545.0, 49.0, 58.0),
 (13339.0, 11.0, 0.0, 692.0, 8032.8, 61.0, 2074.0),
 (13022.0, 14.0, 0.0, 1398.0, 18041.0, 79.0, 558.0),
 (25761.0, 12.0, 0.0, 286.0, 19246.0, 31.0, 825.0),
 (24097.0, 23.0, 0.0, 1402.0, 13042.0, 69.0, 2075.0),
 (26312.0, 11.0, 0.0, 819.0, 8372.8, 48.0, 2076.0),
 (25800.0, 

In [57]:
cnx = mysql.connector.connect (user = "root", 
                               password = "AlumnaAdalab", 
                               host='127.0.0.1',
                               database="talent_transformation")

mycursor = cnx.cursor()

query_insert = "INSERT INTO 3_economic_data (Monthlyrate, Percentsalaryhike, Stockoptionlevel, Dailyrate, Monthlyincome, Hourlyrate, 1_personal_data_Employeenumber) VALUES (%s, %s, %s, %s, %s, %s, %s)"

#variables en el mismo orden que scsv, tantas %s como variables tiene mi csv. 


try: 
    mycursor.executemany(query_insert, lista_tuplas_tabla)
    #ejecuta my query, de que datos? del fichero q estan en mi lista de tuplas q he converido ariba leyendo el csv.) 
    cnx.commit()
    print(mycursor.rowcount, "registros insertados")

except mysql.connector.Error as error:
    print("Error al insertar datos en MySQL:", error)



Error al insertar datos en MySQL: Failed processing format-parameters; Python 'float64' cannot be converted to a MySQL type


In [42]:
df_tabla3.head()

,Monthlyrate,Percentsalaryhike,Stockoptionlevel,Dailyrate,Monthlyincome,Hourlyrate,Employeenumber
0,6462,13,0,684.0,19537.0,51.0,162
1,5678,14,1,699.0,19999.0,65.0,259
2,4933,11,0,532.0,19232.0,58.0,319
3,26703,19,2,359.0,17169.0,82.0,2069
4,7739,12,1,1319.0,4884.6,45.0,2070


In [27]:
df_tabla3.dtypes

Monthlyrate            int64
Percentsalaryhike      int64
Stockoptionlevel       int64
Dailyrate            float64
Monthlyincome        float64
Hourlyrate           float64
Employeenumber         int64
dtype: object

# TABLA 4 Satisfaction

In [ ]:
df_tabla4 = df_rawdata[["Environmentsatisfaction","Jobsatisfaction", "Relationshipsatisfaction", "Performancerating", "Worklifebalance", "Jobinvolvement", "Employeenumber"]]
df_tabla4.head(3)

,Environmentsatisfaction,Jobsatisfaction,Relationshipsatisfaction,Performancerating,Worklifebalance,Jobinvolvement,Employeenumber
0,Not Satisfied,Satisfied,Satisfied,3.0,3.0,High,162
1,Satisfied,Satisfied,Not Satisfied,3.0,3.0,Moderate,259
2,Satisfied,Very Satisfied,Very Satisfied,3.0,3.0,High,319


In [ ]:
lista_tuplas_tabla = []
for x in df_tabla4.values:

    lista_tuplas_tabla.append(tuple(x))
    lista_tuplas_tabla

In [ ]:
cnx = mysql.connector.connect (user = "root", 
                               password = "AlumnaAdalab", 
                               host='127.0.0.1',
                               database="talent_transformation")

mycursor = cnx.cursor()

query_insert = "INSERT INTO 4_satisfaction (Environmentsatisfaction, Jobsatisfaction, Relationshipsatisfaction, Performancerating, Worklifebalance, Jobinvolvement, 1_personal_data_Employeenumber) VALUES (%s, %s,%s, %s, %s, %s, %s)"
#variables en el mismo orden que scsv, tantas %s como variables tiene mi csv. 


try: 
    mycursor.executemany(query_insert, lista_tuplas_tabla)
    #ejecuta my query, de que datos? del fichero q estan en mi lista de tuplas q he converido ariba leyendo el csv.) 
    cnx.commit()
    print(mycursor.rowcount, "registros insertados")

except mysql.connector.Error as error:
    print("Error al insertar datos en MySQL:", error)

1510 registros insertados


In [ ]:
df_tabla4.dtypes

Environmentsatisfaction      object
Jobsatisfaction              object
Relationshipsatisfaction     object
Performancerating           float64
Worklifebalance             float64
Jobinvolvement               object
Employeenumber                int64
dtype: object